# Import library

In [ ]:
### Load data ###
import pandas as pd
from sklearn.model_selection import train_test_split

### Create ImageDataGenerator ###
from keras.preprocessing.image import ImageDataGenerator

# Static

In [ ]:
### Load data ###
PATH_TRAIN = '/content/drive/MyDrive/SANDBOX/DATASET/train_images.csv'
PATH_PREDS = '/content/drive/MyDrive/SANDBOX/DATASET/test_images.csv'

PATH_IMAGE_TRAIN = '/content/drive/MyDrive/SANDBOX/DATASET/train_images'
PATH_IMAGE_PREDS = '/content/drive/MyDrive/SANDBOX/DATASET/test_images'

PATH_MODEL_EFFICIENTNETB7 = '/content/drive/MyDrive/SANDBOX/DATASET/efficientnetb7.h5'

SPLIT_TRAIN_TEST = 0.1
SPLIT_TRAIN_VALIDATION = 0.2
RANDOM_STATE = 2021

### Create ImageDataGenerator ###
X_COL = 'id'
Y_COL = 'class_num'
IMAGE_SIZE = 640
BATCH_SIZE = 1
CLASS_MODE = 'categorical'

### Create model ###
DROPOUT_RATE = 0.5

### Warmup ###
EPOCHS = 100

MONITOR_EARLYSTOPPING = 'val_loss'
MONITOR_MODELCHECKPOINT = 'val_loss'

### Model compile & fit ###
LEARNING_RATE = 1e-5

# Function

## Load data

In [ ]:
def load_data(PATH_TRAIN, PATH_PREDS, SPLIT_TRAIN_TEST, SPLIT_TRAIN_VALIDATION, RANDOM_STATE):
  df = pd.read_csv(PATH_TRAIN)
  df_preds = pd.read_csv(PATH_PREDS)
  
  df['id'] = PATH_IMAGE_TRAIN+'/'+df['id']
  df['class_num'] = df['class_num'].astype(str)

  print('df shape: {0}, df_preds shape: {1}'.format(df.shape, df_preds.shape))

  df_train, df_test = train_test_split(df, test_size=SPLIT_TRAIN_TEST, random_state=RANDOM_STATE)
  df_train, df_val = train_test_split(df_train, test_size=SPLIT_TRAIN_VALIDATION, random_state=RANDOM_STATE)

  print('df_train shape: {0}, df_val shape: {1}, df_test shape: {2}'.format(df_train.shape, df_val.shape, df_test.shape))

  return df_train, df_val, df_test

## Create ImageDataGenerator

In [ ]:
def create_imagedatagenerator(df_train, df_val, df_test, X_COL, Y_COL, IMAGE_SIZE, BATCH_SIZE, CLASS_MODE):
  train_datagen = ImageDataGenerator(featurewise_center=False,
                                    samplewise_center=False,
                                    featurewise_std_normalization=False,
                                    samplewise_std_normalization=False,
                                    zca_whitening=False,
                                    rotation_range=360,
                                    width_shift_range=1.0,
                                    height_shift_range=1.0,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    channel_shift_range=0.0,
                                    fill_mode='nearest',
                                    cval=0.0,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    rescale=1./255.,
                                    preprocessing_function=None,
                                    data_format=None,)
  # train_datagen = ImageDataGenerator(rescale=1./255.,)
  val_datagen = ImageDataGenerator(rescale=1./255.,)
  test_datagen = ImageDataGenerator(rescale=1./255.,)

  train_generator = train_datagen.flow_from_dataframe(df_train, 
                                                      directory=PATH_IMAGE_TRAIN, 
                                                      x_col=X_COL, 
                                                      y_col=Y_COL, 
                                                      target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                                      batch_size=BATCH_SIZE, 
                                                      class_mode=CLASS_MODE)
  val_generator = val_datagen.flow_from_dataframe(df_val, 
                                                  directory=PATH_IMAGE_TRAIN, 
                                                  x_col=X_COL, 
                                                  y_col=Y_COL, 
                                                  target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                                  batch_size=BATCH_SIZE, 
                                                  class_mode=CLASS_MODE, 
                                                  shuffle=False)
  test_generator = test_datagen.flow_from_dataframe(df_test, 
                                                    directory=PATH_IMAGE_TRAIN, 
                                                    x_col=X_COL, 
                                                    y_col=Y_COL, 
                                                    target_size=(IMAGE_SIZE, IMAGE_SIZE), 
                                                    batch_size=BATCH_SIZE, 
                                                    class_mode=CLASS_MODE, 
                                                    shuffle=False)

  # for train_data_batch, train_labels_batch, val_data_batch, val_labels_batch, test_data_batch, test_labels_batch in zip(train_generator, val_generator, test_generator):
  #   print('train data batch shape: {0}, val data batch shape: {1}, test data batch shape: {2}'.format(train_data_batch.shape, val_data_batch.shape, test_data_batch.shape))
  #   print('train labels batch shape: {0}, val labels batch shape: {1}, test labels batch shape: {2}'.format(train_labels_batch.shape, val_labels_batch.shape, test_labels_batch.shape))
  #   break

  return train_generator, val_generator, test_generator

## Create model

In [ ]:
from tensorflow.keras import Input
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Activation
from keras.models import Model

def create_model(IMAGE_SIZE, DROPOUT_RATE):
  input_layer = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3), name='efficientnetb7')
  conv_base = EfficientNetB7(weights='imagenet', input_shape=input_layer, include_top=False)

  ### Transfer learning ###
  for layer in conv_base.layer:
    layer.trainable = False
  
  _ = GlobalAveragePooling2D()(_)
  _ = Dropout(DROPOUT_RATE)(_)
  _ = Dense(2048)(_)
  _ = Activation('relu')(_)
  _ = Dropout(DROPOUT_RATE)(_)
  _ = Dense(4)(_)
  output_layer = Activation('softmax')(_)

  model = Model(input_layer, output_layer)

  model.compile()

  return model

# Main

In [ ]:
### Load data ###
df_train, df_val, df_test = load_data(PATH_TRAIN, PATH_PREDS, SPLIT_TRAIN_TEST, SPLIT_TRAIN_VALIDATION, RANDOM_STATE)

df shape: (1102, 2), df_preds shape: (1651, 1)
df_train shape: (792, 2), df_val shape: (199, 2), df_test shape: (111, 2)


In [ ]:
### Create ImageDataGenerator ###
train_generator, val_generator, test_generator = create_imagedatagenerator(df_train, df_val, df_test, X_COL, Y_COL, IMAGE_SIZE, BATCH_SIZE, CLASS_MODE)

Found 792 validated image filenames belonging to 4 classes.
Found 199 validated image filenames belonging to 4 classes.
Found 111 validated image filenames belonging to 4 classes.


In [ ]:
### Create model ###
model = create_model(IMAGE_SIZE, DROPOUT_RATE)

TypeError: ignored

## Warmup

In [ ]:
!pip install tensorflow_addons
from tensorflow_addons.optimizers import RectifiedAdam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import time

callbacks_list = [EarlyStopping(monitor=MONITOR_EARLYSTOPPING, patience=5, mode='min',),
                  CustomCallback(model, val_generator),
                  ModelCheckpoint(filepath=MODEL_PATH, monitor=MONITOR_MODELCHECKPOINT, save_best_only=True, mode='min',),]

model.compile(optimizer=RectifiedAdam(total_steps=10000, warmup_proportion=0.1, min_lr=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy',],)

model.fit(train_generator,
          epochs=EPOCHS_WARMUP,
          batch_size=BATCH_SIZE,
          callbacks=callbacks_list,
          validation_data=val_generator,)

from keras.models import load_model

model = load_model(MODEL_PATH_EFFICIENTNETB7)

for layer in model.layers:
    layer.trainable = True

## TRAIN

In [ ]:
callbacks_list = [EarlyStopping(monitor=MONITOR_EARLYSTOPPING, patience=5, mode='min',),
                  CustomCallback(model, val_generator),
                  ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-6, verbose=1)
                  ModelCheckpoint(filepath=MODEL_PATH, monitor=MONITOR_MODELCHECKPOINT, save_best_only=True, mode='min',),]

model.compile(optimizer=Adam(lr=LEARNING_RATE), loss='categorical_crossentropy',  metrics=['accuracy'])

model.fit(train_generator,
          epochs=EPOCHS_WARMUP,
          batch_size=BATCH_SIZE,
          callbacks=callbacks_list,
          validation_data=val_generator,)

In [ ]:
### Evaluation ###
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

fig = plt.figure(0)

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'b', color='orange', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.show()

fig = plt.figure(1)

plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'b', color='orange', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from tensorflow_addons.metrics import CohenKappa
from keras.models import load_model

clf = load_model(MODEL_PATH_EFFICIENTNETB7)

import numpy as np
from sklearn.metrics import confusion_matrix 
import matplotlib.pyplot as plt
import seaborn as sns

predicted = np.argmax(clf.predict(test_generator), axis=1)
test_labels = test_generator.classes

cf_matrix = confusion_matrix(test_labels, predicted)

fig = plt.figure(figsize=(16, 8))

sns.heatmap(cf_matrix, annot=True, cmap='Greens', xticklabels=LABEL_NAMES, yticklabels=LABEL_NAMES)

plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.show()

In [ ]:
from sklearn.metrics import cohen_kappa_score, accuracy_score

print('Test cohen kappa score: %.3f' % cohen_kappa_score(np.argmax(clf.predict(test_generator), axis=1), test_generator.classes, weights='quadratic'))
print('Test accuracy score : %.3f' % accuracy_score(np.argmax(clf.predict(test_generator), axis=1), test_generator.classes))

In [ ]:
### Submission ###
import numpy as np
from PIL import Image

preds = []

for image_id in df_preds['id']:
    image = Image.open(TEST_IMAGE_PATH+'/'+image_id)
    image = np.expand_dims(image, axis=0)
    image = image / 255.
    preds.append(np.argmax(clf.predict(image)))

df_preds['label'] = preds

df_preds.to_csv('/content/drive/MyDrive/SANDBOX/DATASET/submission.csv', index=False, header=False)

df_preds.head()